In [508]:
import pandas as pd
from sklearn.svm import SVR
import numpy as np

def mape(y_pred, y_true):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [629]:
dataset = pd.read_csv("../data/preprocessed_input_interpolate_20min_phase1and2_train.csv")
del dataset['date']

# Only select columns which are useful
selected_col = ["hour"]

# Target predict columns
predict_col = "(3, 1, 'tot')"
predict_col_shifted = "predict_shift_6"

# Time shift backward for predict_col
dataset_shift_1 = dataset.shift(periods=-1)[predict_col].fillna(method="ffill")
dataset_shift_2 = dataset.shift(periods=-2)[predict_col].fillna(method="ffill")
dataset_shift_6 = dataset.shift(periods=-6)[predict_col].fillna(method="ffill")

# Add time shifted predict_col to dataset
#dataset = dataset.assign(predict_shift_1=pd.Series(dataset_shift_1).values)
#dataset = dataset.assign(predict_shift_2=pd.Series(dataset_shift_2).values)
dataset = dataset.assign(predict_shift_6=pd.Series(dataset_shift_6).values)
#selected_col.append(predict_col)
#selected_col.append("predict_shift_1")
#selected_col.append("predict_shift_2")
#selected_col.append("predict_shift_6")

training_set = dataset[:-24*3*14]
predict_set = dataset[-24*3*14:-24*3*7][lambda df : ((df.hour >= 6) & (df.hour < 8)) | ((df.hour >= 15) & (df.hour < 17))]

# Prepare potential features to add to the feature set
unused_features = [predict_col, predict_col_shifted]
unused_features.extend(selected_col)
if predict_col == "(1, 0, 'tot')":
    unused_features.extend(["(1, 0, 'cargocar')", "(1, 0, 'etc')", "(1, 0, 'motorcycle')", "(1, 0, 'privatecar')", "(1, 0, 'tot')", "(1, 0, 'unknowncar')"])
elif predict_col == "(1, 1, 'tot')":
    unused_features.extend(["(1, 1, 'cargocar')", "(1, 1, 'etc')", "(1, 1, 'motorcycle')", "(1, 1, 'privatecar')", "(1, 1, 'tot')", "(1, 1, 'unknowncar')"])
elif predict_col == "(2, 0, 'tot')":
    unused_features.extend(["(2, 0, 'cargocar')", "(2, 0, 'etc')", "(2, 0, 'motorcycle')", "(2, 0, 'privatecar')", "(2, 0, 'tot')", "(2, 0, 'unknowncar')"])
elif predict_col == "(3, 0, 'tot')":
    unused_features.extend(["(3, 0, 'cargocar')", "(3, 0, 'etc')", "(3, 0, 'motorcycle')", "(3, 0, 'privatecar')", "(3, 0, 'tot')", "(3, 0, 'unknowncar')"])
elif predict_col == "(3, 1, 'tot')":
    unused_features.extend(["(3, 1, 'cargocar')", "(3, 1, 'etc')", "(3, 1, 'motorcycle')", "(3, 1, 'privatecar')", "(3, 1, 'tot')", "(3, 1, 'unknowncar')"])
potential_features = list(training_set.columns)
potential_features = [x for x in potential_features if x not in unused_features]

In [630]:
clf = SVR(C=1.0, epsilon=0.1)
clf.fit(training_set[selected_col], training_set[predict_col_shifted])
base_score = mape(clf.predict(predict_set[selected_col]), predict_set[predict_col_shifted])
print(base_score)

28.136529714119828


In [631]:
while True:
    next_feature = None
    for col in potential_features:
        selected_col.append(col)
        clf.fit(training_set[selected_col], training_set[predict_col_shifted])
        score = mape(clf.predict(predict_set[selected_col]), predict_set[predict_col_shifted])
        if base_score > score:
            base_score = score
            next_feature = col
        selected_col.pop()
    if next_feature is not None:
        selected_col.append(next_feature)
        print(next_feature, base_score)
        potential_features.remove(next_feature)
    else:
        break
        
print(selected_col)

is_holiday 23.819959098878904
['hour', 'is_holiday']


In [632]:
training_set = dataset[:-24*3*7]
predict_set = dataset[-24*3*7:][lambda df : ((df.hour >= 6) & (df.hour < 8)) | ((df.hour >= 15) & (df.hour < 17))]
clf.fit(training_set[selected_col], training_set[predict_col_shifted])
final_score = mape(clf.predict(predict_set[selected_col]), predict_set[predict_col_shifted])
print(final_score)

18.28061238129334
